### 必要なパッケージをインポート

#### .envファイル
```yaml
OPENAI_API_KEY=sk-************************
```

In [2]:
import openai,os,requests,json,re,bs4
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

### Prompts

InitialプロンプトとRefineプロンプトを作成した。\
不要な「,」を含んで、json.loads()でパースできない場合があったため、以下を追加した。\
（これでもたまにパースできないJsonを返してくる。。\
LangChainのOutput Parserを使いたいが、リストや入れ子構造があるJsonに対する方法がわからない。\
Pydantic Parserだと、英語で書かないとプロンプトが文字化けする。）

```
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。
```

また、Refineプロンプトでは、必要な場合のみ修正するようにした。
ドキュメントの最初に重要な情報が多く、最後に近づくにつれ関連コンテンツなど、メインコンテンツと関係ないものが多くなるという仮定を入れている。

```
[Text]をもとに、[Data]について新しい情報がある場合のみ[Data]を修正してください。\
[Text]が[Data]に関連した情報を提供しない場合、[Data]をそのまま出力してください。
```

In [4]:
initial_prompt = """\
Input:
{text}

Instruction:
[Input]はある{data_type}のWebページのテキストの一部です。
"""

refine_prompt = """\
Data:
{data}

Input:
{text}

Instruction:
[Input]はある{data_type}のWebページのテキストの一部です。\
あなたのタスクは、[Data]を[Input]をもとにRefineすることです。\
[Input]が[Data]に関連した追加情報を提供しない場合、[Data]をそのまま出力してください。
"""

### ヘルパー関数

メイン関数で使う関数などを定義した。

`chunk_size`などは、現在はえいやで決めている。\
指示プロンプトと出力Jsonフォーマットのトークン数に基づき、トークン数制約ぎりぎりになるように調整することでチャンク数を抑えられるため、\
今後クラス化して使うときに調整機能をつけようと思う。

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 8194,
    chunk_overlap  = 512,
    length_function = len,
)
chat = ChatOpenAI(temperature=0.)

def load_doc(url):
    response = requests.get(url)
    with open('temp.html','w') as f:
        f.write(response.text)
    loader = UnstructuredHTMLLoader("temp.html")
    data = loader.load()
    return data

# def get_raw_html(url):
#     response = requests.get(url)
#     return response.text

# def get_comressed_html(url):
#     html = get_raw_html(url)
#     soup = bs4.BeautifulSoup(html, 'html.parser')

### メイン関数

まずはWebページのテキストをLangChainのドキュメントとし、チャンクに分割する。\
１つ目のチャンクにはInitialプロンプト、２つ目以降の各チャンクにはRefineプロンプトを適用する。\
最後にソースURLをデータに加えて終了。

In [21]:
def extact_json_from_url(url,function,debug=False):
    doc = load_doc(url)
    docs = text_splitter.split_documents(doc)
    # text = get_raw_html(url)
    # docs = text_splitter.create_documents([text])
    data = {}
    if debug:
        print('docs:',len(docs))
    for i,doc in enumerate(docs):
        text = doc.page_content
        if debug:
            print(i+1,'-'*30)
            print(text.replace('\n','').replace(' ','')[:100],'...')
            print('...',text.replace('\n','').replace(' ','')[-100:])
        if i == 0:
            messages = [
                {
                    'role': 'user',
                    "content": initial_prompt.format(text=text,data_type='レシピ')
                }
            ]
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages=messages,
                functions=[function]
            )
            message = response["choices"][0]["message"]
            if 'function_call' in message:
                if debug:
                    print(message['function_call']['arguments'])
                data = json.loads(message['function_call']['arguments'])
        else:
            messages = [
                {
                    'role': 'user',
                    "content": refine_prompt.format(text=text,data_type='レシピ',data=json.dumps(data,indent=2,ensure_ascii=False))
                }
            ]
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages=messages,
                functions=[function]
            )
            message = response["choices"][0]["message"]
            if 'function_call' in message:
                if debug:
                    print(message['function_call']['arguments'])
                data = json.loads(message['function_call']['arguments'])
        if debug:
            print(json.dumps(data,ensure_ascii=False))
    data['url'] = url
    return data

### Example: レシピ

```json
{
    "name": "レシピ名",
    "description": "レシピの説明",
    "cooking_time": "int: minutes",
    "appliances": [
        "str: 調理器具"
    ],
    "serves": "int: 人前",
    "ingredients": [
        {
            "name": "str: 材料名",
            "amount": "str: 分量表記",
        }
    ],
    "steps": {
        "str: 手順番号":"str: 手順の説明"
    }
}
```

In [7]:
function = {
    "name": "get_recipe",
    "description": "レシピをJson形式で返す",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string", "description": "レシピ名"
            },
            "description": {
                "type": "string", "description": "レシピの説明"
            },
            "cooking_time": {
                "type": "number", "description": "調理時間"
            },
            "appliances": {
                "type": "array", "description": "使用機器のリスト",
                "items": {
                    "type": "string", "description": "使用機器"
                }
            },
            "serves": {
                "type": "number", "description": "人前"
            },
            "ingredients": {
                "type": "object", "description": "材料のリスト",
                "properties": {
                    "name": {
                        "type": "string", "description": "材料名"
                    },
                    "amount": {
                        "type": "string", "description": "材料の分量表記。例：200g, 大さじ1, 2個, etc",
                    }
                }
            },
            "instructions": {
                "type": "array", "description": "調理手順のリスト",
                "items": {
                    "type": "string", "description": "調理手順"
                }
            },
        },
        # "required": ["name","appliances","ingredients","instructions"],
    }
}

In [8]:
# url = "https://park.ajinomoto.co.jp/recipe/card/705645/"
url = "https://www.lettuceclub.net/recipe/dish/14360/"

In [9]:
data = extact_json_from_url(url,function,debug=True)
print(json.dumps(data,indent=2,ensure_ascii=False))

docs: 1
1 ------------------------------
レタスクラブレシピカレーおからカレーおから時短でラク275kcal1.5g15min井原裕子さんのレシピしっとり煮上げるのがおいしさのコツ豚こま切れ肉おからパプリカピーマン玉ねぎその他豆腐料理Face ...
... 肉条件を細かく指定するラクレシピならレタスクラブ今日の夕飯のおかず&献立を探すならレタスクラブで！基本の定番料理から人気料理まで、日々のへとへとから解放されるプロ監修の簡単レシピ33349品をご紹介！
{
  "name": "カレーおから",
  "description": "時短でラク",
  "cooking_time": 15,
  "serves": 2,
  "ingredients": [
    { "name": "豚こま切れ肉", "amount": "100g" },
    { "name": "おから", "amount": "100g" },
    { "name": "赤パプリカ", "amount": "1/2個" },
    { "name": "ピーマン", "amount": "2個" },
    { "name": "玉ねぎ", "amount": "1/2個" },
    { "name": "トマトケチャップ", "amount": "大さじ2" },
    { "name": "カレー粉", "amount": "大さじ1〜1.5" },
    { "name": "酒", "amount": "大さじ1" },
    { "name": "塩", "amount": "小さじ1/3" },
    { "name": "水", "amount": "130ml" },
    { "name": "サラダ油", "amount": null }
  ],
  "instructions": [
    "パプリカは縦３等分に切り、横半分に切る。ピーマンは縦半分に切り、さらに縦横それぞれ半分に切る。玉ねぎは縦３等分のくし形に切り、横半分に切る。",
    "フライパンに油大さじ1/2を熱し、豚肉を炒め、肉の色が変わったらを１加えて炒め合わせる。全体に油がまわったら、煮汁の材料を加えて混ぜ、煮立ったらふたをして

### Example: イベント

```json
{
    "title": "str: イベント名",
    "organizer": "str: 主催者",
    "date": ["str: 日付。形式はYYYY/MM/DD"],
    "location": "str: 開催場所",
    "description": "str: イベント概要。400字以内",
    "schedule": ["str: スケジュール。形式はYYYY/MM/DD HH:MM~HH:MM（補足）"],
}
```

In [18]:
function ={
    "name": "get_event",
    "description": "AgendaのリストをJson形式で返す",
    "parameters": {
        "type": "object",
        "properties": {
            "agenda": {
                "type": "array",
                "description": "Agendaのリスト",
                "items": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "タイトル"
                        },
                        "organizer": {
                            "type": "string",
                            "description": "スピーカー"
                        },
                        "day": {
                            "type": "string",
                            "description": "開始時刻。形式はHH:MM",
                        },
                        "time": {
                            "type": "string",
                            "description": "開始時刻。形式はHH:MM",
                        },
                        "speaker": {
                            "type": "array",
                            "description": "スピーカーのリスト",
                            "items": {
                                "type": "string",
                                "description": "スピーカー名"
                            }
                        },
                        "descrption": {
                            "type": "string",
                            "description": "イベント概要。400字以内"
                        }
                    }
                }
            }
        }
    }
}

In [19]:
# url = "https://food-innovation.co/sksjapan/sksj2023/"
# url = "https://www.nagoya-info.jp/event/detail/404/"
url = 'https://food-innovation.co/sksjapan/sksj2023/#agenda'

In [22]:
data = extact_json_from_url(url,function,debug=True)
print(json.dumps(data,indent=2,ensure_ascii=False))

docs: 23
1 ------------------------------
NEWS&TOPICSOVERVIEWHIGHLIGHTSAGENDASPEAKERSPARTNERSEnglishNEWS&TOPICSOVERVIEWHIGHLIGHTSAGENDASPEAKER ...
... 大阪・関西万博の大阪ヘルスケアパビリオンの出展参加者として、2050年の食の未来を描いている。本セッションでは、このコンソーシアムが目指していることについて解説する。桐原慎也(シグマクシス)18:20
{}
2 ------------------------------
BenHarris(Fresco)KevinYu(SideChef)RobinLiss(SUVIE)[Moderator]MichaelWolf(TheSpoon)17:45Break18:00Ses ...
... queCulinaryCenter)NabilaRodriguez(BasqueCulinaryCenter)[Moderator]沢田明大(東京建物/TokyoFoodInstitute)18:00
{
  "agenda": [
    {
      "title": "08:15 - Opening",
      "organizer": "田中 宏隆 (シグマクシス)",
      "day": "Day 1",
      "time": "8:15",
      "speaker": ["田中 宏隆 (シグマクシス)", "住 朋享 (シグマクシス)"],
      "descrition": "10. Opening"
    },
    {
      "title": "09:00 - Session",
      "organizer": "Omid Bakhshandeh (Verneek), Kevin Yu (SideChef), [Moderator]Michael Wolf (The Spoon)",
      "day": "Day 1",
      "time": "9:00",
      "speaker": ["Omid Bakhshandeh (Verneek)", "Kevin Yu (SideChef)", "[Modera

JSONDecodeError: Expecting value: line 1 column 1 (char 0)